In [1]:
# Setup Environment
import os
import sys
import asyncio
import importlib
from pathlib import Path

# Environment detection and setup
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    os.system('pip install openai-agents pandas python-dotenv')
    os.system('git clone https://github.com/heresandyboy/oxford-ai-2025-group1-full-code.git')
    os.chdir('/content/oxford-ai-2025-group1-full-code')
    
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
else:
    from dotenv import load_dotenv
    load_dotenv()
    
    # Find project root
    current_path = Path().resolve()
    project_root = current_path
    while project_root.parent != project_root:
        if (project_root / "pyproject.toml").exists():
            break
        project_root = project_root.parent
    sys.path.insert(0, str(project_root))

# Verify setup
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("OpenAI API key required")

print("✅ Environment ready")


✅ Environment ready


In [2]:
# Import Dependencies and Load Data
from agents import Runner, trace
from src.logistics_agents.agents.agent_01_threshold_monitor.agent import InventoryThresholdMonitor
from src.logistics_agents.utils.data_loader import create_inventory_context
from src.logistics_agents.utils.notebook_logging import run_agent_with_capture, display_logs, display_agent_config, clear_logs

# Load real inventory data (100 items from CSV)
project_root = Path.cwd() if IN_COLAB else project_root
csv_path = project_root / "data" / "final_customer_location_aligned.csv"
context = create_inventory_context(str(csv_path), region="Multi-region")

print(f"📊 Loaded {len(context.items)} items ({len(context.items_below_threshold)} below threshold)")


[07/02/25 18:43:42] INFO     📝 Logging to: logs\logistics_agents_20250702_184342_heres.log    ]8;id=447957;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py\logging_config.py]8;;\:]8;id=996016;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py#87\87]8;;\

📊 Loaded 100 items (21 below threshold)


In [3]:
# BASELINE - Run initial agent
agent = InventoryThresholdMonitor()

with trace(workflow_name="Baseline"):
    result = await run_agent_with_capture(
        agent.agent,
        "Analyze inventory thresholds and prioritize items needing restocking.",
        context
    )

# Show agent configuration after capture
print("\n🤖 BASELINE AGENT CONFIGURATION:")
display_agent_config()

print("\n📋 BASELINE RESULT:")
print(result.final_output)

# Show detailed execution logs
display_logs()


                    INFO     🤖 InventoryThresholdMonitor initialized                            ]8;id=853499;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\agent_runner.py\agent_runner.py]8;;\:]8;id=393666;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\agent_runner.py#43\43]8;;\

🔄 Running agent...


[07/02/25 18:43:47] INFO     🔧 ThresholdChecker executing...                                 ]8;id=83172;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py\logging_config.py]8;;\:]8;id=833136;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py#211\211]8;;\

                    INFO     🔧 ThresholdChecker completed                                    ]8;id=639534;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py\logging_config.py]8;;\:]8;id=396531;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py#225\225]8;;\

                    INFO     🔧 PriorityClassifier executing...                               ]8;id=812923;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py\logging_config.py]8;;\:]8;id=679494;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py#211\211]8;;\

                    INFO     🔧 PriorityClassifier completed                                  ]8;id=146248;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py\logging_config.py]8;;\:]8;id=634256;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py#225\225]8;;\

✅ Agent execution complete

🤖 BASELINE AGENT CONFIGURATION:
🤖 AGENT CONFIGURATION
**Agent Name**: InventoryThresholdMonitor
**Model**: gpt-4o-mini
**Tools Available**: check_inventory_thresholds, classify_item_priority

📋 **AGENT INSTRUCTIONS/PROMPT**:
----------------------------------------
You are an Inventory Threshold Monitor agent in a multi-agent logistics system.

**Learning Focus**: Demonstrate @function_tool usage and basic data analysis patterns.

**Your Role**: Monitor inventory levels and identify items below reorder thresholds.

**Available Tools**:
- check_inventory_thresholds: Check which items are below their reorder points
- classify_item_priority: Classify urgent items by priority (HIGH/MEDIUM/LOW)

**Agent Pattern**: You work as part of an "Agents as Tools" system where:
1. You analyze inventory data to find threshold violations
2. You classify the urgency of items needing restocking
3. Your analysis helps other agents (route planning, quantity calculation) make dec

In [4]:
# ITERATION 1 - Reload agent and test changes
import src.logistics_agents.agents.agent_01_threshold_monitor.agent as agent_module
importlib.reload(agent_module)

agent_v1 = agent_module.InventoryThresholdMonitor()

with trace(workflow_name="Iteration1"):
    result_v1 = await run_agent_with_capture(
        agent_v1.agent,
        "Analyze inventory thresholds and prioritize items needing restocking.",
        context
    )

# Show updated agent configuration after capture
print("\n🔄 ITERATION 1 AGENT CONFIGURATION:")
display_agent_config()

print("\n📋 ITERATION 1 RESULT:")
print(result_v1.final_output)

# Show detailed execution logs
display_logs()


[07/02/25 18:43:54] INFO     🤖 InventoryThresholdMonitor initialized                            ]8;id=820206;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\agent_runner.py\agent_runner.py]8;;\:]8;id=859447;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\agent_runner.py#43\43]8;;\

🔄 Running agent...


[07/02/25 18:43:56] INFO     🔧 ThresholdChecker executing...                                 ]8;id=287925;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py\logging_config.py]8;;\:]8;id=603128;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py#211\211]8;;\

                    INFO     🔧 ThresholdChecker completed                                    ]8;id=744669;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py\logging_config.py]8;;\:]8;id=484044;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py#225\225]8;;\

                    INFO     🔧 PriorityClassifier executing...                               ]8;id=295824;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py\logging_config.py]8;;\:]8;id=874210;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py#211\211]8;;\

                    INFO     🔧 PriorityClassifier completed                                  ]8;id=245034;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py\logging_config.py]8;;\:]8;id=850006;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py#225\225]8;;\

✅ Agent execution complete

🔄 ITERATION 1 AGENT CONFIGURATION:
🤖 AGENT CONFIGURATION
**Agent Name**: InventoryThresholdMonitor
**Model**: gpt-4o-mini
**Tools Available**: check_inventory_thresholds, classify_item_priority

📋 **AGENT INSTRUCTIONS/PROMPT**:
----------------------------------------
You are an Inventory Threshold Monitor agent in a multi-agent logistics system.

**Learning Focus**: Demonstrate @function_tool usage and basic data analysis patterns.

**Your Role**: Monitor inventory levels and identify items below reorder thresholds.

**Available Tools**:
- check_inventory_thresholds: Check which items are below their reorder points
- classify_item_priority: Classify urgent items by priority (HIGH/MEDIUM/LOW)

**Agent Pattern**: You work as part of an "Agents as Tools" system where:
1. You analyze inventory data to find threshold violations
2. You classify the urgency of items needing restocking
3. Your analysis helps other agents (route planning, quantity calculation) make 

In [5]:
# ITERATION 2 - Test improved formatting
importlib.reload(agent_module)
agent_v2 = agent_module.InventoryThresholdMonitor()

with trace(workflow_name="Iteration2"):
    result_v2 = await run_agent_with_capture(
        agent_v2.agent,
        "Generate an actionable restocking report with specific recommendations and timelines.",
        context
    )

# Show updated agent configuration after capture
print("\n🔄 ITERATION 2 AGENT CONFIGURATION:")
display_agent_config()

print("\n📋 ITERATION 2 RESULT:")
print(result_v2.final_output)

# Show detailed execution logs
display_logs()


[07/02/25 18:44:03] INFO     🤖 InventoryThresholdMonitor initialized                            ]8;id=979392;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\agent_runner.py\agent_runner.py]8;;\:]8;id=402492;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\agent_runner.py#43\43]8;;\

🔄 Running agent...


[07/02/25 18:44:04] INFO     🔧 ThresholdChecker executing...                                 ]8;id=85926;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py\logging_config.py]8;;\:]8;id=889907;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py#211\211]8;;\

                    INFO     🔧 ThresholdChecker completed                                    ]8;id=795001;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py\logging_config.py]8;;\:]8;id=885578;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py#225\225]8;;\

                    INFO     🔧 PriorityClassifier executing...                               ]8;id=321682;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py\logging_config.py]8;;\:]8;id=250260;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py#211\211]8;;\

                    INFO     🔧 PriorityClassifier completed                                  ]8;id=466481;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py\logging_config.py]8;;\:]8;id=720988;file://C:\code\ai\mine\shared\oxford-ai-2025-group1-full-code\src\logistics_agents\utils\logging_config.py#225\225]8;;\

✅ Agent execution complete

🔄 ITERATION 2 AGENT CONFIGURATION:
🤖 AGENT CONFIGURATION
**Agent Name**: InventoryThresholdMonitor
**Model**: gpt-4o-mini
**Tools Available**: check_inventory_thresholds, classify_item_priority

📋 **AGENT INSTRUCTIONS/PROMPT**:
----------------------------------------
You are an Inventory Threshold Monitor agent in a multi-agent logistics system.

**Learning Focus**: Demonstrate @function_tool usage and basic data analysis patterns.

**Your Role**: Monitor inventory levels and identify items below reorder thresholds.

**Available Tools**:
- check_inventory_thresholds: Check which items are below their reorder points
- classify_item_priority: Classify urgent items by priority (HIGH/MEDIUM/LOW)

**Agent Pattern**: You work as part of an "Agents as Tools" system where:
1. You analyze inventory data to find threshold violations
2. You classify the urgency of items needing restocking
3. Your analysis helps other agents (route planning, quantity calculation) make 

In [6]:
# INTEGRATION - Create tool from agent
threshold_tool = agent_v2.agent.as_tool(
    tool_name='InventoryThresholdMonitor',
    tool_description='Monitor inventory thresholds and prioritize urgent restocking needs'
)

print(f"✅ Tool created successfully")
print(f"📝 Tool type: {type(threshold_tool)}")
print(f"📋 This agent can now be used as a tool by the orchestrator")

# Show integration code for orchestrator
print("\n📄 ORCHESTRATOR INTEGRATION:")
print("""
# In Agent 05 (Orchestrator):
threshold_agent = InventoryThresholdMonitor()
threshold_tool = threshold_agent.agent.as_tool(
    tool_name='InventoryThresholdMonitor',
    tool_description='Monitor inventory thresholds and prioritize urgent restocking'
)

orchestrator = Agent(
    model='gpt-4o',
    instructions='Coordinate supply chain optimization...',
    tools=[threshold_tool, route_tool, restock_tool, consolidator_tool]
)

# When orchestrator runs, it can call this agent as a tool:
# - Automatically receives inventory context
# - Gets priority analysis results
# - Uses for decision making in supply chain workflow
""")


✅ Tool created successfully
📝 Tool type: <class 'agents.tool.FunctionTool'>
📋 This agent can now be used as a tool by the orchestrator

📄 ORCHESTRATOR INTEGRATION:

# In Agent 05 (Orchestrator):
threshold_agent = InventoryThresholdMonitor()
threshold_tool = threshold_agent.agent.as_tool(
    tool_name='InventoryThresholdMonitor',
    tool_description='Monitor inventory thresholds and prioritize urgent restocking'
)

orchestrator = Agent(
    model='gpt-4o',
    instructions='Coordinate supply chain optimization...',
    tools=[threshold_tool, route_tool, restock_tool, consolidator_tool]
)

# When orchestrator runs, it can call this agent as a tool:
# - Automatically receives inventory context
# - Gets priority analysis results
# - Uses for decision making in supply chain workflow

